In [32]:
import numpy as np

In [33]:
#input data
X=np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
#output data
Y=np.array([[0],[1],[1],[0]])




In [67]:
class SimpleNN:
    def __init__(self, inputSize, hiddenLayers, outputSize):
        #synapses
        self._syn0=np.random.random((inputSize,hiddenLayers))
        self._syn1=np.random.random((hiddenLayers,outputSize))
    
    def nonlin(self, x, deriv=False):
        if deriv:
            return x*(1-x)
        return 1.0/(1.0+np.exp(-x))

    def run(self, X, Y, n_iter=60000):
        for i in range(n_iter):
            l0 = X
            l1 = nonlin(np.dot(l0,self._syn0))
            l2 = nonlin(np.dot(l1,self._syn1))
            
            l2_error = Y-l2
            if (i+1) % 10000==0 or i ==0:
                print("Iter=",i+1, "Error=",np.mean(np.abs(l2_error)))
            
            l2_delta = l2_error * nonlin(l2, deriv=True)
            l1_error = l2_delta.dot(self._syn1.T)
            l1_delta = l1_error * nonlin(l1, deriv=True)
            
            #update
            self._syn1 += l1.T.dot(l2_delta)
            self._syn0 += l0.T.dot(l1_delta)
        print("----")
        print(X)
        print(Y)
        print(l2)
        print(np.argmax(l2, axis=1))
    
    def print_syns(self):
        print(self._syn0)
        print(self._syn1)
    

In [68]:
snn = SimpleNN(3,4,1)
snn.run(X,Y)


Iter= 1 Error= 0.496566336035
Iter= 10000 Error= 0.0124170290708
Iter= 20000 Error= 0.00854169596933
Iter= 30000 Error= 0.00689498561136
Iter= 40000 Error= 0.005931564767
Iter= 50000 Error= 0.00528147275821
Iter= 60000 Error= 0.00480530519781
----
[[0 0 1]
 [0 1 1]
 [1 0 1]
 [1 1 1]]
[[0]
 [1]
 [1]
 [0]]
[[ 0.00138257]
 [ 0.99451964]
 [ 0.99447956]
 [ 0.00683785]]
[0 0 0 0]


In [38]:
snn.print_syns()

[[ 7.17462703  3.14277846  0.96706835  5.98528201]
 [ 6.63360455  4.25357779  4.51013659 -4.28574902]
 [-2.8765724  -5.62249783 -3.61743333  2.29305974]]
[[ 13.24941049]
 [ -7.47081579]
 [ -6.70372468]
 [ -6.70078631]]


In [2]:
import gzip
import json
import numpy as np
from collections import defaultdict

workpath="/data/jmoreno/Datasets/signalmedia/"

f=gzip.open(workpath+'signalmedia-1k.jsonl.gz','rb')


docs=[json.loads(s.decode('utf-8')) for s in f.readlines()]
titres=[x['title'] for x in docs]
content=[x['content'] for x in docs]
print(len(titres))

def sentense2cleanTokens(sent):
    sent = sent.lower()
    sent = "".join([x if x.isalpha() else " " for x in sent])
    sent = " ".join(sent.split())
    return sent


cleantitres=[sentense2cleanTokens(x) for x in titres]

from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer(min_df=2, stop_words='english')
tfsk = tf_vectorizer.fit_transform(cleantitres)

1000


In [48]:
ivocab = tf_vectorizer.get_feature_names()
vocab = {}
for i,w in enumerate(ivocab):
    vocab[w]=i
vocab

{'years': 1066,
 'hughes': 460,
 'sister': 886,
 'jailed': 496,
 'seven': 873,
 'round': 828,
 'sitting': 888,
 'jennifer': 499,
 'premiere': 728,
 'showing': 882,
 'professional': 746,
 'dubai': 289,
 'photo': 696,
 'nuclear': 647,
 'pro': 741,
 'car': 132,
 'official': 656,
 'unique': 1007,
 'ruled': 834,
 'flowers': 365,
 'closing': 177,
 'great': 419,
 'suspicious': 945,
 'awarded': 66,
 'loan': 549,
 'jobs': 502,
 'deal': 244,
 'frankfurt': 378,
 'steps': 919,
 'conference': 197,
 'recycling': 786,
 'brian': 112,
 'opener': 662,
 'predictions': 727,
 'goes': 410,
 'parents': 676,
 'foreign': 372,
 'schedule': 847,
 'boost': 102,
 'daniel': 234,
 'outside': 669,
 'hiring': 444,
 'live': 545,
 'surprise': 941,
 'aussie': 59,
 'art': 51,
 'plc': 713,
 'target': 956,
 'products': 745,
 'brad': 107,
 'revealed': 810,
 'cup': 227,
 'welcomes': 1044,
 'dogs': 273,
 'paul': 686,
 'blatter': 91,
 'carry': 135,
 'registration': 792,
 'begin': 82,
 'ford': 370,
 'incident': 470,
 'plays': 71

In [54]:
vocabSize = len(ivocab)
[x for x in cleantitres[0].split() if x in vocab]

['worcester', 'breakfast', 'club', 'veterans', 'gives', 'orders']

In [64]:
doc0 = [vocab[x] for x in cleantitres[0].split() if x in vocab]
print(doc0)
print(doc0[:len(doc0)-1])
print(doc0[1:])

[1059, 110, 179, 1020, 405, 666]
[1059, 110, 179, 1020, 405]
[110, 179, 1020, 405, 666]


In [65]:
vectors = np.array([[vocab[x] for x in cleantitres[0].split() if x in vocab]]).reshape(-1)
one_hot_vectors = np.eye(vocabSize)[vectors]
print(one_hot_vectors[0])
print(sum(one_hot_vectors[0]))
print(one_hot_vectors[0][1059])

[ 0.  0.  0. ...,  0.  0.  0.]
1.0
1.0


In [76]:
simplewordembeddings = SimpleNN(vocabSize,5,vocabSize)
simplewordembeddings.run(one_hot_vectors[:len(one_hot_vectors)-1],one_hot_vectors[1:],n_iter=200000)

Iter= 1 Error= 0.811247849538
Iter= 10000 Error= 0.500000190629
Iter= 20000 Error= 0.500000131117
Iter= 30000 Error= 0.500000101904
Iter= 40000 Error= 0.500000083907
Iter= 50000 Error= 0.500000071493
Iter= 60000 Error= 0.500000062318
Iter= 70000 Error= 0.500000055212
Iter= 80000 Error= 0.500000049516
Iter= 90000 Error= 0.500000044829
Iter= 100000 Error= 0.500000040891
Iter= 110000 Error= 0.500000037524
Iter= 120000 Error= 0.500000034605
Iter= 130000 Error= 0.500000032043
Iter= 140000 Error= 0.500000029769
Iter= 150000 Error= 0.500000027733
Iter= 160000 Error= 0.500000025893
Iter= 170000 Error= 0.500000024217
Iter= 180000 Error= 0.50000002268
Iter= 190000 Error= 0.50000002126
Iter= 200000 Error= 0.500000019941
----
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ..., 

In [72]:
print(np.argmax(one_hot_vectors[1:], axis=1))

[ 110  179 1020  405  666]


In [77]:
simplewordembeddings = SimpleNN(vocabSize,5,vocabSize)
simplewordembeddings.run(one_hot_vectors[:len(one_hot_vectors)-1],one_hot_vectors[1:],n_iter=200000)

Iter= 1 Error= 0.816596176245
Iter= 10000 Error= 0.500000224025
Iter= 20000 Error= 0.500000147028
Iter= 30000 Error= 0.500000109171
Iter= 40000 Error= 0.500000085472
Iter= 50000 Error= 0.500000068433
Iter= 60000 Error= 0.500000054804
Iter= 70000 Error= 0.500000042702
Iter= 80000 Error= 0.500000030526
Iter= 90000 Error= 0.500000015966
Iter= 100000 Error= 0.499999993097
Iter= 110000 Error= 0.499999928593
Iter= 120000 Error= 0.00749440447818
Iter= 130000 Error= 0.00428838434892
Iter= 140000 Error= 0.00358504821551
Iter= 150000 Error= 0.00323612904287
Iter= 160000 Error= 0.00301833513451
Iter= 170000 Error= 0.00286590173212
Iter= 180000 Error= 0.00275156320721
Iter= 190000 Error= 0.00266171016609
Iter= 200000 Error= 0.00258869199182
----
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]

In [78]:
print(np.argmax(one_hot_vectors[1:], axis=1))

[ 110  179 1020  405  666]


In [81]:
print(ivocab[110])
print(simplewordembeddings._syn0[110])

breakfast
[-14.88983354   8.56395537 -15.17251454 -14.3849577  -14.9929134 ]


In [82]:
print(ivocab[1020])
print(simplewordembeddings._syn0[1020])

veterans
[-14.65993327  10.46939782 -14.8943817  -16.02371326 -15.44001003]


In [1]:
from __future__ import print_function
import collections
import math
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [14]:
words=sum([sentense2cleanTokens(x).split() for x in content],[])
print(len(words))
words[:20]

415857


['veterans',
 'saluted',
 'worcester',
 's',
 'first',
 'ever',
 'breakfast',
 'club',
 'for',
 'ex',
 'soldiers',
 'which',
 'won',
 'over',
 'hearts',
 'minds',
 'and',
 'bellies',
 'the',
 'worcester']

In [15]:
vocabulary_size = 10000

def build_dataset(words):
    count = [['UNK', -1]]
    #add most frequent (without taking account eats
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count = unk_count + 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data, count, dictionary, reverse_dictionary



data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Most common words (+UNK) [['UNK', 26326], ('the', 22122), ('to', 11320), ('and', 11156), ('of', 9891)]
Sample data [2697, 0, 8985, 8, 55, 401, 3055, 654, 7, 1730]


In [16]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

In [17]:
print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 5
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])



data: ['veterans', 'UNK', 'worcester', 's', 'first', 'ever', 'breakfast', 'club']

with num_skips = 2 and skip_window = 1:
    batch: ['breakfast', 'breakfast', 'club', 'club', 'for', 'for', 'ex', 'ex']
    labels: ['ever', 'club', 'for', 'breakfast', 'club', 'ex', 'soldiers', 'for']

with num_skips = 4 and skip_window = 2:
    batch: ['club', 'club', 'club', 'club', 'for', 'for', 'for', 'for']
    labels: ['ex', 'ever', 'for', 'breakfast', 'breakfast', 'soldiers', 'club', 'ex']


In [18]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

In [19]:
with graph.as_default(), tf.device('/cpu:0'):
    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    # Variables.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    softmax_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(
        tf.nn.sampled_softmax_loss(weights=softmax_weights, biases=softmax_biases, inputs=embed,
                               labels=train_labels, num_sampled=num_sampled, num_classes=vocabulary_size))
    # Optimizer.
    # Note: The optimizer will optimize the softmax_weights AND the embeddings.
    # This is because the embeddings are defined as a variable quantity and the
    # optimizer's `minimize` method will by default modify all variable quantities 
    # that contribute to the tensor it is passed.
    # See docs on `tf.train.Optimizer.minimize()` for more details.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(
        normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))


In [21]:
num_steps = 30001

with tf.Session(graph=graph) as session:
    saver = tf.train.Saver()
    tf.global_variables_initializer().run()
    print('Initialized')
    average_loss = 0
    for step in range(num_steps):
        batch_data, batch_labels = generate_batch(
          batch_size, num_skips, skip_window)
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += l
        if step % 2000 == 0:
            if step > 0:
                average_loss = average_loss / 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print('Average loss at step %d: %f' % (step, average_loss))
            average_loss = 0
        # note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8 # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k+1]
                log = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log = '%s %s,' % (log, close_word)
                print(log)
    final_embeddings = normalized_embeddings.eval()
    #saver.save(session, 'modelW2V.vec')	
    #writer_ = tf.summary.FileWriter('./', session.graph)

Initialized
Average loss at step 0: 6.711872
Nearest to are: athletic, bonds, thrown, andujar, worldwide, processed, exactly, states,
Nearest to only: likely, lists, inaugural, indecent, devopssummit, griffin, range, pressing,
Nearest to would: refuse, cs, auditor, ringgit, chip, interior, iran, tensas,
Nearest to years: clinton, windows, accordingly, size, morning, pdp, materials, functional,
Nearest to our: acquiring, rising, warrant, remark, longtime, collar, considerably, emergence,
Nearest to there: musical, this, hypertension, possibility, glyphosate, door, retailers, radio,
Nearest to of: gross, complements, browning, chinese, inspectors, addition, visibility, feelings,
Nearest to get: reference, herein, join, protect, expect, applying, authentic, documented,
Nearest to what: young, poseidon, convenience, workforce, usaconsultants, scrap, mouth, twenty,
Nearest to company: corporate, killings, darwin, applying, signal, holyrood, commit, shield,
Nearest to back: sole, standardise